In [198]:
import difflib

# def replace_mention(

e = 'Professor Farnsworth'
f = 'Farnsworth'
matcher = difflib.SequenceMatcher(None, e, f)
match = matcher.find_longest_match(0, len(e), 0, len(f))
match

Match(a=10, b=0, size=10)

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')
# Due to a bug, this language model doesn't contain
# stop words, so we fix that here.
for word in nlp.Defaults.stop_words:
    lex = nlp.vocab[word]
    lex.is_stop = True

In [411]:
from spacy import displacy
def show_parse(sentence, style='dep'):
    parsed = nlp(sentence)
    svg = displacy.render(parsed, style=style)
    with open('sentence.svg', 'w') as f:
        f.write(svg)

    displacy.render(parsed, style=style, jupyter=True)
    
    return parsed

In [273]:
big_text = """A self-described mad scientist, the Professor is a 
          senile, amoral, deranged, and unpredictable old man 
          (160 years old as of "A Clone of My Own"), with very 
          thick glasses and a gift for creating doomsday devices 
          and atomic supermen. He has put at least one parallel 
          universe in peril with his inventions and visited dozens 
          more (see The Farnsworth Parabox).

          The Professor teaches at Mars University and has worked for 
          Momcorp on several occasions, but spends most of his time 
          inventing ridiculous devices and sending the Planet Express 
          delivery crew on suicide missions. While at Momcorp, he fell 
          in love with the CEO, Mom, only to leave her and Momcorp when 
          she decided to weaponize his "Q.T. McWhiskers" toy. What he is 
          a professor of is never explicitly stated. In the episode Mars 
          University when asked what he is teaching, he responds: 
          "The same thing I teach every semester, the mathematics of 
          quantum neutrino fields. I made up the title so no student 
          would dare take it"; however, this declaration has not precluded 
          the professor from demonstrating mastery of whatever field of science 
          is convenient for a given episode's plot, as shown in Bender's Big Score 
          when he proclaims, "I can wire anything directly into anything! I am the 
          Professor!", proceeding to link Hermes' disembodied head to the ship's 
          computer. Approximately 100 years before the series' timeline, he taught a
          young (not yet Professor) Wernstrom, whom Farnsworth regarded as a prized 
          student. After he returned a pop quiz to Wernstrom with a grade of A-minus 
          (for poor penmanship), the two became bitter rivals (established in "A Big 
          Piece of Garbage")."""

big_text = text.replace('\n', '')


In [10]:
article_title = 'Professor Farnsworth'
article_title = article_title.lower()

relevant_subjects = []
doc = nlp(text)
for sent in doc.sents:
    for token in sent:
        if token.dep_ == 'nsubj' and token.text.lower() in article_title:
            relevant_subjects.append(token)

In [144]:
type(parsed_sent)

spacy.tokens.doc.Doc

In [136]:
parsed_sent = nlp(sent)
for token in parsed_sent:
    print(token.text, token.dep_)
' '.join(token.text for token in parsed_sent)   

I nsubj
'm ROOT
obi compound
- punct
wan compound
- punct
ki attr
. punct


"I 'm obi - wan - ki ."

In [157]:
not True

False

In [178]:
sent = "The heinous Professor Farnsworth treacherously killed Obi-wan."

In [179]:
tokens = nlp(sent)

In [180]:
show_parse(sent)

The heinous Professor Farnsworth treacherously killed Obi-wan.

In [346]:
def tokens_to_str(tokens, spaces_before_punct=False):
    tokens = list(tokens)
    strings = []
    for i, token in enumerate(tokens):
        is_first_word = i == 0
        
        cur_not_string = not isinstance(token, str)
        last_two_not_str = i >= 1 and cur_not_string and not isinstance(tokens[i - 1], str)
        last_three_not_str = i >= 2 and last_two_not_str and not isinstance(tokens[i - 2], str)
        is_dash_after_compound = last_two_not_str and token.text == '-' and tokens[i - 1].dep_ == 'compound'
        is_token_after_dash = last_three_not_str and tokens[i - 1].text == '-' and tokens[i - 2].dep_ == 'compound'
        cur_is_punct = cur_not_string and (token.is_punct or token.dep_ == 'case')
        
        skip_space = is_first_word or is_dash_after_compound or is_token_after_dash or (cur_is_punct and not spaces_before_punct)
        
        if not isinstance(token, str):
            token = token.text
        
        token = token.strip()
        if not token:
            continue
        
        if not skip_space:
            strings.append(' ')
            
        strings.append(token)
        
    return ''.join(strings)

tokens_to_str(tokens)

'The heinous Professor Farnsworth treacherously killed Obi-wan.'

In [339]:
# parsed = nlp(sent)
# noun = parsed[5]

# print(list(noun.head.subtree))
import re

def filter_sent(sent_text, target_name, cap_first_word=True):
    
    remove_patterns = [r'\(.+?\)', r'\[.+?\]']
    for pattern in remove_patterns:
        sent_text = re.sub(pattern, '', sent_text)
    
    parsed_sent = nlp(sent_text)
    
    # Extract
    subjects_of_interest = []
    for token in parsed_sent:
        if token.dep_ == 'nsubj' and token.text.lower() in target_name.lower():
            subjects_of_interest.append(token)
    
    output = []
    for subj in subjects_of_interest:
        selected_tokens = []
        subtree_heads = list(subj.head.children)
        
        for token in subj.head.subtree:
            if token == subj.head or token.head == subj.head:
                selected_tokens.append(token)
        
        if selected_tokens[0].dep_ == 'dobj':
            dobj = selected_tokens.pop(0)
            verb_index = selected_tokens.index(subj.head)
            selected_tokens.insert(verb_index + 1, dobj)
        
        final_tokens = []   
        for token in selected_tokens:
            if token == subj:
                subj_str = tokens_to_str(token.subtree)
                matcher = difflib.SequenceMatcher(None, target_name, subj_str)
                match = matcher.find_longest_match(0, len(target_name), 0, len(subj_str))
                subj_str = subj_str[:match.b] + '<TARGET>' + subj_str[match.b + match.size:]
                final_tokens.append(subj_str)
            elif token.dep_ == 'attr':
                final_tokens.extend(token.subtree)
            elif token == subj.head:
                final_tokens.append(token.text)
            elif token.dep_ == 'dobj':
                final_tokens.append('<OBJECT>')
         
        final_tokens.append(nlp('.')[0])
        as_str = tokens_to_str(final_tokens)
        if cap_first_word:
            split = as_str.split(' ')
            if '<' not in split[0]:
                as_str = ' '.join([split[0].capitalize()] + split[1:])
        output.append(as_str)
            
    return output
    
filter_sent("The heinous Professor Farnsworth treacherously killed the monster of New York while traveling east.", 'Professor Farnsworth')

['The heinous <TARGET> killed <OBJECT>.']

In [400]:
# parsed = nlp(sent)
# noun = parsed[5]

# print(list(noun.head.subtree))
import re

def filter_sent(sent_text, target_name, cap_first_word=True):
    
    remove_patterns = [r'\(.+?\)', r'\[.+?\]']
    for pattern in remove_patterns:
        sent_text = re.sub(pattern, '', sent_text)
    
    parsed_sent = nlp(sent_text)
    
    # Extract
    subjects_of_interest = []
    for token in parsed_sent:
        if token.dep_ == 'nsubj' and token.text.lower() in target_name.lower():
            subjects_of_interest.append(token)
    
    output = []
    for subj in subjects_of_interest:
        selected_tokens = []
        subtree_heads = list(subj.head.children)
        
        for token in subj.head.subtree:
            if token == subj.head or token.head == subj.head:
                selected_tokens.append(token)
        
        if selected_tokens[0].dep_ == 'dobj':
            dobj = selected_tokens.pop(0)
            verb_index = selected_tokens.index(subj.head)
            selected_tokens.insert(verb_index + 1, dobj)
        
        basic_filters = [(lambda x: x.pos_ == 'PROPN', '<PROPN>')]
        
        final_tokens = []   
        for token in selected_tokens:
            if token == subj:
                subj_str = tokens_to_str(token.subtree)
                matcher = difflib.SequenceMatcher(None, target_name, subj_str)
                match = matcher.find_longest_match(0, len(target_name), 0, len(subj_str))
                subj_str = subj_str[:match.b] + '<TARGET>' + subj_str[match.b + match.size:]
                final_tokens.append(subj_str)
            elif token.dep_ == 'attr':
                filtered_sub = filter_subtree(token, basic_filters)
                final_tokens.extend(filtered_sub)
            elif token == subj.head:
                final_tokens.append(token.text)
            elif token.dep_ == 'dobj':
                final_tokens.append('<OBJECT>')
         
        final_tokens.append(nlp('.')[0])
        as_str = tokens_to_str(final_tokens)
        if cap_first_word:
            split = as_str.split(' ')
            if '<' not in split[0]:
                as_str = ' '.join([split[0].capitalize()] + split[1:])
        output.append(as_str)
            
    return output
    
filter_sent("The heinous Professor Farnsworth treacherously killed the monster of New York while traveling east.", 'Professor Farnsworth')

['The heinous <TARGET> killed <OBJECT>.']

In [ ]:
fast_nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])

In [423]:
for i in range(10000):
    fast_nlp('hi')

KeyboardInterrupt: 

In [ ]:
def recur_replace(head):
    if head.pos_ == 'PROPN':
        return ['<OBJECT>']
    return 

In [359]:
def replace_proper_nouns(subtree):
    subtree = list(subtree)
    output = subtree[:]
    for token in subtree:
        if token.pos_ == 'PROPN':
            for subtoken in token.subtree:
                print(output)
                print(subtoken)
#                 if subtoken != token:
#                 output.remove(subtoken)
#             output[output.index(token)] = '<OBJECT>'
    return output

sent = """Avatar Aang is a fictional character and the protagonist of Nickelodeon's animated television series Avatar: The Last Airbender."""
parsed = nlp(sent)
replace_proper_nouns(parsed[1].head.subtree)

[Avatar, Aang, is, a, fictional, character, and, the, protagonist, of, Nickelodeon, 's, animated, television, series, Avatar, :]
Avatar
[Avatar, Aang, is, a, fictional, character, and, the, protagonist, of, Nickelodeon, 's, animated, television, series, Avatar, :]
Avatar
[Avatar, Aang, is, a, fictional, character, and, the, protagonist, of, Nickelodeon, 's, animated, television, series, Avatar, :]
Aang
[Avatar, Aang, is, a, fictional, character, and, the, protagonist, of, Nickelodeon, 's, animated, television, series, Avatar, :]
Nickelodeon
[Avatar, Aang, is, a, fictional, character, and, the, protagonist, of, Nickelodeon, 's, animated, television, series, Avatar, :]
's
[Avatar, Aang, is, a, fictional, character, and, the, protagonist, of, Nickelodeon, 's, animated, television, series, Avatar, :]
Avatar


[Avatar,
 Aang,
 is,
 a,
 fictional,
 character,
 and,
 the,
 protagonist,
 of,
 Nickelodeon,
 's,
 animated,
 television,
 series,
 Avatar,
 :]

In [398]:
target = 'Professor Farnsworth'
sent= """A self-described mad scientist, the Professor is a 
  senile, amoral, deranged, and unpredictable old man with very 
  thick glasses and a gift for creating doomsday devices 
  and atomic supermen."""
sent = sent.replace('\n', '')
filter_sent(big_text, target)

['The <TARGET> is a senile, amoral, deranged, and unpredictable old man.',
 'The <TARGET> teaches.',
 '<TARGET> regarded <OBJECT>.']

In [418]:
sent = """Aang is the last surviving Airbender, a monk of the Air Nomads' Southern Air Temple. He was created by Jim Dale."""
show_parse(sent, style='ent')
parsed = nlp(anng)
for token in parsed:
    print(token, token.ent_id, token.ent_type_)

Avatar 0 
Aang 0 PERSON
( 0 
安昂 0 
Ān 0 
Áng 0 
) 0 
is 0 
a 0 
fictional 0 
character 0 
and 0 
the 0 
protagonist 0 
of 0 
Nickelodeon 0 ORG
's 0 
animated 0 
television 0 
series 0 
Avatar 0 WORK_OF_ART
: 0 WORK_OF_ART
The 0 
Last 0 
Airbender 0 PRODUCT
( 0 
created 0 
by 0 
Michael 0 PERSON
Dante 0 PERSON
DiMartino 0 PERSON
and 0 
Bryan 0 PERSON
Konietzko 0 PERSON
) 0 
, 0 
voiced 0 
by 0 
Zach 0 PERSON
Tyler 0 PERSON
Eisen 0 PERSON
. 0 


 0 CARDINAL
Aang 0 PERSON
is 0 
the 0 
last 0 
surviving 0 
Airbender 0 PRODUCT
, 0 
a 0 
monk 0 
of 0 
the 0 ORG
Air 0 ORG
Nomads 0 ORG
' 0 ORG
Southern 0 FAC
Air 0 FAC
Temple 0 FAC
. 0 
He 0 
is 0 
an 0 
incarnation 0 
of 0 
the 0 
" 0 
Avatar 0 PRODUCT
" 0 
, 0 
the 0 
spirit 0 
of 0 
light 0 
and 0 
peace 0 
manifested 0 
in 0 
human 0 
form 0 
. 0 
As 0 
the 0 
Avatar 0 PRODUCT
, 0 
Aang 0 PERSON
controls 0 
all 0 
four 0 CARDINAL
elements 0 
and 0 
is 0 
tasked 0 
with 0 
keeping 0 
the 0 ORG
Four 0 ORG
Nations 0 ORG
at 0 
peace 0 
. 0 
At 

In [340]:
anng = """Avatar Aang is a fictional character and the protagonist of Nickelodeon's animated television series Avatar: The Last Airbender (created by Michael Dante DiMartino and Bryan Konietzko), voiced by Zach Tyler Eisen.

Aang is the last surviving Airbender, a monk of the Air Nomads' Southern Air Temple. He is an incarnation of the "Avatar", the spirit of light and peace manifested in human form. As the Avatar, Aang controls all four elements and is tasked with keeping the Four Nations at peace. At 12 years old, Aang is the series' reluctant hero, spending a century in suspended animation before joining new friends Katara and Sokka on a quest to master the elements and save their world from the imperialist Fire Nation.

Aang's character has appeared in other media, such as trading cards,[1][2] video games,[3][4] T-shirts,[5] and web comics.[6] Aang has also been portrayed by Noah Ringer in the feature film The Last Airbender,[7] and voiced by D.B. Sweeney in the sequel animated series The Legend of Korra."""

In [ ]:
"As the Avatar, Aang controls all four elements and is tasked with keeping the Four Nations at peace."

In [401]:
filter_sent(anng, 'Avatar Aang')

['<TARGET> is a fictional character and the protagonist of <PROPN> animated television series <PROPN>.',
 '<TARGET> is <PROPN>.',
 '<TARGET> controls <OBJECT>.',
 "<TARGET> is the series' reluctant hero."]

In [387]:
for token in nlp(sent):
    print(token, token.dep_, )

Avatar compound
Aang nsubj
is ROOT
a det
fictional amod
character attr
and cc
the det
protagonist conj
of prep
Nickelodeon poss
's case
animated amod
television compound
series pobj
Avatar appos
: punct
The det
Last amod
Airbender ROOT
. punct


In [391]:
def foo():
    pass
hasattr(foo, '__call__')

True

In [394]:
def filter_subtree(head, rules):
    for rule, replacement in rules:
        if rule(head):
            if callable(replacement):
                replacement = replacement(head)
            return [replacement]
    if len(list(head.children)) == 0:
        return [head]
    output = []
    for child in head.lefts:
        output.extend(get_subtree(child, rules))
    output.append(head)
    for child in head.rights:
        output.extend(get_subtree(child, rules))
    
    return [token for token in output if token]

rules = [
    (lambda x: x.dep_ == 'prep', None),

]

get_subtree(head, rules)    

[Avatar, Aang, is, a, fictional, character, and, the, protagonist, :]

In [390]:
def get_subtree(head, dead_ends, exceptions=None):
    if head.dep_ in dead_ends:
        return ['<OBJECT>']
    elif len(list(head.children)) == 0:
        return [head]
    output = []
    for child in head.lefts:
        output.extend(get_subtree(child, dead_ends))
    output.append(head)
    for child in head.rights:
        output.extend(get_subtree(child, dead_ends))
    
    return output
    

parsed = nlp(sent)

head = parsed[2]
get_subtree(head, ['pobj'])

[Avatar,
 Aang,
 is,
 a,
 fictional,
 character,
 and,
 the,
 protagonist,
 of,
 '<OBJECT>',
 :]

In [31]:
sent= """Jack killed the golden goose."""
sent = sent.replace('\n', '')
sent = nlp(sent)
for word in sent:
    if word.dep_ == 'nsubj' or word.dep_ == 'dobj':
        for e in word.subtree:
            print(e, e.dep_, e.pos)
        print()

Jack nsubj 95

the det 89
golden amod 83
goose dobj 91

